In [11]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Consts 
k1 = 10**(-1.47)
k2 = 10**(-2.59)
k3 = 10**(-3.76)
k4 = 10**(-10.329)
yppm = 415e-06

In [9]:
# Eq Consts Only
ph = 5.5 # Avoid if possible
cb = k1 * yppm * 1
cd = k2 * cb
ce = 10**(-ph)
cf = k3 * cd / ce
cg = k4 * cf / ce

ct = cb + cf + cg
print('xCO2 {:.3g}, xHCO3 {:.3g}, xCO3 {:.3g}'.format(cb/ct, cf/ct, cg/ct))
phs = np.logspace(-14, 
plt.plot(

xCO2 0.876, xHCO3 0.124, xCO3 1.83e-06
